<a href="https://colab.research.google.com/github/alice9547/covid-19_model/blob/ksh/corona19_regional_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자치구별 코로나 확진자 예측

## [ 서울 열린 데이터 광장 ] 
### 서울특별시 코로나19 자치구별 확진자 발생동향 API
https://data.seoul.go.kr/dataList/OA-20470/S/1/datasetView.do

💡 본 프로그램은 https://github.com/kairess/corona-virus-prediction 의 응용임을 밝힙니다.

In [ ]:
# 필요 라이브러리 불러오기
import requests # API를 사용하기 위한 모듈
from bs4 import BeautifulSoup # 텍스트 형태의 HTML과 XML에서 데이터를 추출하기 위한 라이브러리
import pandas as pd # 텍스트 입출력 및 데이터프레임 가공이 가능 한 라이브러리

import plotly.graph_objs as go # 그래프 라이브러리
import plotly.offline as py
from fbprophet import Prophet # 시계열 예측 라이브러리
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import json, urllib.request
import requests
import csv

import numpy as np #행렬이나 다차원 배열을 처리할때 사용하는 라이브러리

#### 공공데이터 포털 API

In [ ]:
#ServiceKey = 'dypF53MOyB5SvOwH2UtPxnT4vEggzFRRQrSIAV%2BZW3%2FNurg6Mo4Zl5DxsNjqCDHxBVE5hcadyB1%2FxU92iwPtgA%3D%3D'
#url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey="
#api_url = url + ServiceKey

#### 서울데이터열린광장 API

In [ ]:
ServiceKey = '74766d63646e61793639476a424f51' # API 인증키
url = "http://openAPI.seoul.go.kr:8088/"+ ServiceKey + "/json/TbCorona19CountStatusJCG/1/550/"
# api_url의 "/xml/TbCorona19CountStatusJCG/1/5/"에서 '5'부분은 가져올 데이터 수. 조절하여 가져올 수 있다.

In [ ]:
data = urllib.request.urlopen(url).read()

#받아온 json 데이터 특정하기
output = json.loads(data)
output = output['TbCorona19CountStatusJCG']
output = output['row']

output_file = 'api_regional.csv'

#CSV파일 제작

try :
    with open(output_file, 'w', newline='', encoding='UTF-8') as csvfile :
        writer = csv.DictWriter(csvfile, output[0].keys())
        writer.writeheader()
        for data in output:
            writer.writerow(data)
except :
    print("Error")

In [ ]:
data = pd.read_csv('api_regional.csv')# pandas데이터 프레임을 이용하여 csv파일을 읽어옴
data.tail()# 데이터가 잘 출력 되는지 5행만 출력해 살펴보며 확인

,JCG_DT,JONGNO,JONGNOADD,JUNGGU,JUNGGUADD,YONGSAN,YONGSANADD,SEONGDONG,SEONGDONGADD,GWANGJIN,GWANGJINADD,DDM,DDMADD,JUNGNANG,JUNGNANGADD,SEONGBUK,SEONGBUKADD,GANGBUK,GANGBUKADD,DOBONG,DOBONGADD,NOWON,NOWONADD,EP,EPADD,SDM,SDMADD,MAPO,MAPOADD,YANGCHEON,YANGCHEONADD,GANGSEO,GANGSEOADD,GURO,GUROADD,GEUMCHEON,GEUMCHEONADD,YDP,YDPADD,DONGJAK,DONGJAKADD,GWANAK,GWANAKADD,SEOCHO,SEOCHOADD,GANGNAM,GANGNAMADD,SONGPA,SONGPAADD,GANGDONG,GANGDONGADD,ETC,ETCADD,WORK_DTTM
545,2020.05.23.10,18,0,8,0,38,0,22,0,13,0,34,0,17,0,28,0,9,0,16,0,28,0,30,0,22,0,24,0,26,0,32,0,36,0,13,0,26,0,38,0,54,0,42,1,72,0,45,0,21,1,50,1,NaN
546,2020.05.22.10,18,0,8,0,38,0,22,0,13,1,34,0,17,0,28,0,9,0,16,0,28,1,30,0,22,0,24,0,26,0,32,0,36,0,13,0,26,0,38,0,54,1,41,0,72,0,45,0,20,0,49,0,NaN
547,2020.05.21.10,18,0,8,0,38,0,22,0,12,0,34,0,17,0,28,0,9,0,16,1,27,0,30,0,22,0,24,0,26,1,32,0,36,0,13,0,26,0,38,1,53,0,41,0,72,0,45,0,20,0,49,1,NaN
548,2020.05.20.10,18,0,8,0,38,4,22,0,12,0,34,0,17,0,28,0,9,1,15,1,27,0,30,0,22,0,24,0,25,0,32,0,36,0,13,0,26,0,37,0,53,0,41,1,72,1,45,0,20,1,48,0,NaN
549,2020.05.19.10,18,0,8,0,34,0,22,0,12,0,34,0,17,0,28,1,8,0,14,0,27,0,30,0,22,0,24,0,25,2,32,1,36,1,13,0,26,1,37,0,53,0,40,0,71,0,45,1,19,0,48,0,NaN


In [ ]:
pickGu = input("지역구 = ")

try :
    dc = data.loc[  : ,  ['JCG_DT', pickGu ] ]#필요한 데이터는 날짜와 일일확진자수 이므로 이둘을 추출한다.
    dc = dc.reset_index().rename(columns={ pickGu : 'daily_confirmed'}) #일일 확진자로  이름을 바꾼다.
except :
    print("잘못된 지역구")

지역구 = YONGSANADD


In [ ]:
dc.head()#값이 잘나오는지 헤더 출력으로  확인

,index,JCG_DT,daily_confirmed
0,0,2021.11.24.00,29
1,1,2021.11.23.00,10
2,2,2021.11.22.00,22
3,3,2021.11.21.00,25
4,4,2021.11.20.00,26


In [ ]:

#그래프를 그려보자 (Line plotly)
fig = go.Figure()
fig['layout']['xaxis']['autorange'] = "reversed"

fig.add_trace(
    go.Scatter(
        x=dc.JCG_DT,
        y=dc.daily_confirmed ,
        name='daily confirmed in Seoul '
    )
)

fig

In [ ]:
m = Prophet(
    changepoint_prior_scale=0.5,
    changepoint_range=0.95, 
    yearly_seasonality=False,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='multiplicative'
)

m.fit(dc_prophet)

future = m.make_future_dataframe(periods=7)
forecast = m.predict(future)

fig = plot_plotly(m, forecast)
py.iplot(fig)